<a href="https://colab.research.google.com/github/elichen/aoc2021/blob/main/Day_19_Beacon_Scanner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
input = '''--- scanner 0 ---
404,-588,-901
528,-643,409
-838,591,734
390,-675,-793
-537,-823,-458
-485,-357,347
-345,-311,381
-661,-816,-575
-876,649,763
-618,-824,-621
553,345,-567
474,580,667
-447,-329,318
-584,868,-557
544,-627,-890
564,392,-477
455,729,728
-892,524,684
-689,845,-530
423,-701,434
7,-33,-71
630,319,-379
443,580,662
-789,900,-551
459,-707,401

--- scanner 1 ---
686,422,578
605,423,415
515,917,-361
-336,658,858
95,138,22
-476,619,847
-340,-569,-846
567,-361,727
-460,603,-452
669,-402,600
729,430,532
-500,-761,534
-322,571,750
-466,-666,-811
-429,-592,574
-355,545,-477
703,-491,-529
-328,-685,520
413,935,-424
-391,539,-444
586,-435,557
-364,-763,-893
807,-499,-711
755,-354,-619
553,889,-390

--- scanner 2 ---
649,640,665
682,-795,504
-784,533,-524
-644,584,-595
-588,-843,648
-30,6,44
-674,560,763
500,723,-460
609,671,-379
-555,-800,653
-675,-892,-343
697,-426,-610
578,704,681
493,664,-388
-671,-858,530
-667,343,800
571,-461,-707
-138,-166,112
-889,563,-600
646,-828,498
640,759,510
-630,509,768
-681,-892,-333
673,-379,-804
-742,-814,-386
577,-820,562

--- scanner 3 ---
-589,542,597
605,-692,669
-500,565,-823
-660,373,557
-458,-679,-417
-488,449,543
-626,468,-788
338,-750,-386
528,-832,-391
562,-778,733
-938,-730,414
543,643,-506
-524,371,-870
407,773,750
-104,29,83
378,-903,-323
-778,-728,485
426,699,580
-438,-605,-362
-469,-447,-387
509,732,623
647,635,-688
-868,-804,481
614,-800,639
595,780,-596

--- scanner 4 ---
727,592,562
-293,-554,779
441,611,-461
-714,465,-776
-743,427,-804
-660,-479,-426
832,-632,460
927,-485,-438
408,393,-506
466,436,-512
110,16,151
-258,-428,682
-393,719,612
-211,-452,876
808,-476,-593
-575,615,604
-485,667,467
-680,325,-822
-627,-443,-432
872,-547,-609
833,512,582
807,604,487
839,-516,451
891,-625,532
-652,-548,-490
30,-46,-14'''.split('\n')

In [ ]:
input = open("input.txt").readlines()
input = [x.rstrip() for x in input]

In [ ]:
scanners = []
i = 0
while i < len(input):
    i += 1
    l = []
    while i < len(input) and input[i] != "":
        x,y,z = map(int,input[i].split(','))
        l.append((x,y,z))
        i += 1
    scanners.append(l)
    i += 1

In [ ]:
import math

In [ ]:
def dist(p1, p2):
  return math.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2+(p1[2]-p2[2])**2)

In [ ]:
d = []
for s in scanners:
    m = {}
    for i in range(len(s)):
        for j in range(i+1,len(s)):
            m[dist(s[i],s[j])] = (i,j)
    d.append(m)

In [ ]:
from collections import Counter

In [ ]:
import numpy as np

In [ ]:
def transform_point(point, facing_direction, rotation):
    x, y, z = point

    # transform based on facing direction
    if facing_direction == 'n':
        x, y, z = x, y, z
    elif facing_direction == 's':
        x, y, z = -x, -y, z
    elif facing_direction == 'e':
        x, y, z = y, -x, z
    elif facing_direction == 'w':
        x, y, z = -y, x, z
    elif facing_direction == 'u':
        x, y, z = x, -z, y
    elif facing_direction == 'd':
        x, y, z = x, z, -y

    # transform based on rotation
    if rotation == 90:
        x, z = z, -x
    elif rotation == 180:
        x, z = -x, -z
    elif rotation == 270:
        x, z = -z, x

    return (x, y, z)

In [ ]:
def calc_offset(points_list):
  min_x = min([x for x,y,z in points_list])
  min_y = min([y for x,y,z in points_list])
  min_z = min([z for x,y,z in points_list])
  return (min_x, min_y, min_z)

In [ ]:
def shift_offset(points_list, offset):
  ox,oy,oz = offset
  return [(x-ox,y-oy,z-oz) for x,y,z in points_list]

In [ ]:
def calc_match(i,j):
  intersect = set(d[i].keys()).intersection(set(d[j].keys()))
  if len(intersect) >= 66:
    ii = {scanners[i][d[i][x][0]] for x in intersect}.union({scanners[i][d[i][x][1]] for x in intersect})
    ij = {scanners[j][d[j][x][0]] for x in intersect}.union({scanners[j][d[j][x][1]] for x in intersect})
    oi = calc_offset(ii)
    for dir in 'nsewud':
      for rot in [0,90,180,270]:
        new_list = {transform_point(p, dir, rot) for p in ij}
        offset = tuple([x-y for x,y in zip(calc_offset(new_list),oi)])
        if set(shift_offset(new_list,offset)) == ii:
          return((dir,rot,offset))
  return None

In [ ]:
covered = {0}
sl = []
while len(covered) < len(scanners):
  new_covered = covered.copy()
  for i in covered:
    for j in set(range(len(scanners))) - covered:
      ret = calc_match(i,j)
      if ret:
        # print(i,j,ret)
        nl = [transform_point(p, ret[0], ret[1]) for p in scanners[j]]
        nl = shift_offset(nl, ret[2])
        scanners[j] = nl
        new_covered.add(j)
        sl.append(ret[2])
  covered = new_covered

In [ ]:
len({elem for s in scanners for elem in s})

442

In [ ]:
def manhattan_distance(point1, point2):
    x1, y1, z1 = point1
    x2, y2, z2 = point2
    return abs(x1 - x2) + abs(y1 - y2) + abs(z1 - z2)

In [ ]:
max_dist = 0
for i in range(len(sl)):
  for j in range(i,len(sl)):
    max_dist = max(max_dist,manhattan_distance(sl[i],sl[j]))
max_dist

11079